# Tabelas de espalhamento (tabelas hash)

Uma tabela hash permite a implementação da operação de busca com um número de operações esperado que
não depende do tamanho da tabela, $O(1)$. Embora o custo de pior caso ainda seja proporcional ao
tamanho da tabela, $O(n)$. Em geral, uma tabela hash possui dois componentes principais: um vetor de
<i>buckets</i> e uma função <i>hash</i> (função de espalhamento).

## Vetor de <i>buckets</i>
O vetor de <i>buckets</i> de uma tabela <i>hash</i> é um vetor $A$ de tamanho $N$ em que cada célula de $A$
é uma coleção (<i>bucket</i>) de pares chave-valor. O inteiro $N$ define o número de <i>buckets</i>
do vetor. 

Caso as chaves fossem inteiros bem distribuídos no intervalo $[0, N-1]$, então o vetor de <i>buckets</i>
seria suficiente para a tabela <i>hash</i>. Nesse caso, uma entrada $e$ com chave $k$ seria inserida no <i>bucket</i>
$A[k]$. Caso as chaves sejam inteiros únicos no intervalo $[0, N-1]$, então, cada <i>bucket</i> armazena
no máximo uma entrada. Então, buscas, inserções e remoções no vetor de <i>buckets</i> são $O(1)$. No
entanto, há duas desvantagens:

- Se $N$ for muito maior do que o número de entradas, então há um grande desperdício de memória.
- As chaves devem ser necessariamente inteiras, o que nem sempre é viável. 

Devido a essas duas desvantagens, o vetor de <i>buckets</i> é utilizado em conjunto com alguma função de
espalhamento.

## Função de espalhamento (função <i>hash</i>)
    
Uma função hash $h$ mapeia cada chave $k$ em um inteiro no intervalo $[0, N-1]$, em que $N$ é o
tamanho do vetor de <i>buckets</i>. Com isso, é possível aplicar a função <i>hash</i> para chaves arbitrárias,
não necessariamente inteiras. Assim, uma entrada com chave $k$ é armazenada no <i>bucket</i> $A[h(k)]$. 

Caso haja duas entradas com chaves $k_{1}$ e $k_{2}$ tal que $h(k_{1}) = h(k_{2})$, então tais
entradas estão mapeadas para o mesmo <i>bucket</i> e diz-se que ocorreu uma colisão. Se o <i>bucket</i> em questão
não conseguir armazenar mais de um elemento, então não é possível armazenar ambos os elementos
na mesma tabela hash. Há maneiras de lidar com a colisão, mas a primeira estratégia é evitá-las. 

Uma "boa" função <i>hash</i>:
- Mapeia as chaves do mapa de forma a minimizar as colisões.
- É fácil (rápida) de computar. 

A avaliação de uma função hash é vista como duas ações: 

1) Mapeamento da chave $k$ em um inteiro (código hash ou <i>hash code</i>).

2) Mapeamento do inteiro em um dos índices, $[0, N-1]$, do vetor de <i>buckets</i> (função de
   compressão).

### Código <i>hash</i>

Essa primeira ação consiste em mapear uma chave arbitrária $k$ em um valor inteiro. Esse inteiro não
precisa estar no intervalo $[0, N-1]$ e pode, inclusive, ser negativo. 

Os códigos <i>hash</i> descritos a seguir são baseados na hipótese de que o número de <i>bits</i> de cada tipo é
conhecido.

#### Convertendo para um inteiro

Caso o tipo de dado seja representado usando no máximo o mesmo número de <i>bits</i> da representação de
inteiros dos códigos <i>hash</i>, então, é possível simplesmente utilizar a representação inteira dos
<i>bits</i> como código para o dado. 

Para tipos maiores, em que a representação utiliza o dobro de
<i>bits</i> de um <i>inteiro</i>, por exemplo, uma possibilidade é rebaixar o valor para <i>int</i>. No entanto, nesse caso, metade
da informação é perdida, o que pode aumentar consideravelmente o número de colisões. Um melhoria
consiste em somar as representações inteiras dos <i>bits</i> mais significativos com a dos bits menos
significativos. 

De fato, a abordagem de somar as componentes pode ser estendida para objetos cuja a representação
binária pode ser vista como uma tupla $x = (x_{0}, x_{1}, \ldots, x_{m-1})$. Assim, o código <i>hash</i>
de $x$ pode ser definido como $\displaystyle \sum_{i=0}^{m-1}x_{i}$. 

#### Códigos hash polinomiais

O código apresentado anteriormente não é adequado para strings e outros objetos em que a ordem dos
elementos da tupla $(x_{0}, x_{1}, \ldots, x_{k-1})$ é significativa. Por exemplo, no caso das
strings <i>temp01</i> e <i>temp10</i> haveria colisão, também haveria colisão nas strings <i>stop</i>, <i>tops</i>,
<i>pots</i> e <i>spot</i>. Uma alternativa, que considera a ordem dos elementos $x_{i}$ consiste em escolher
uma constante não nula $a \neq 1$ e utilizar

$$
x_{0}a^{k-1} + x_{1}{a}^{k-2} + \ldots + x_{k-2}a + x_{k-1}
$$

como valor do código <i>hash</i>.

Alguns estudos sugerem que 33, 37, 39 e 41 são boas escolhas para $a$ quando trabalhando com strings
de palavras em inglês. Uma lista com 50000 palavras produziu menos de 7 colisões em cada um dos
casos. 

#### Códigos hash com deslocamento cíclico

Uma variação dos códigos polinomiais consiste em substituir a multiplicação por um deslocamento
(<i>shift</i>) cíclico de um determinado número de <i>bits</i>. Por exemplo, assumindo uma palavra de 32-bits de
comprimento, um <i>shift</i> cíclico de 5 <i>bits</i> pode ser obtido com uma operação lógica <i>ou</i> <i>bit</i> a <i>bit</i>
entre um <i>shift</i> para a esquerda de 5 bits e um a direita de 27 bits.

In [8]:
def hash_code(s):
    mask = (1 << 32) - 1
    h=0
    for character in s:
        h = (h << 5 & mask) | (h >> 27)
        h += ord(character)
    return h

hash_code("stop"), hash_code("tops"), hash_code("spot")

(3890768, 3918451, 3886676)

### Códigos <i>hash</i> em Python

O mecanismo padrão para computar códigos <i>hash</i> em Python é uma função embutida com assinatura <i>hash(x)</i> que retorna um inteiro, a ser utilizado como código <i>hash</i>. No entanto, apenas tipos imutáveis podem ser "hasheados" em Python. Isso mantém o código <i>hash</i> do objeto inalterado durante seu ciclo de vida. Dentre os tipos imutáveis estão <i>int</i>, <i>float</i>, <i>str</i> e <i>tuplas</i>

Instâncias de classes definidas pelo usuário são consideradas não "hasheáveis" por padrão. No entanto, uma função que computa o código <i>hash</i> pode ser definida. O código retornado deve refletir a imutabilidade dos atributos da instância da classe. É comum retornar um código <i>hash</i> baseado nos códigos <i>hash</i> dos atributos imutáveis da instância.

É importante que caso a classe defina equivalência entre os objetos através do método <i>\__eq\__</i>, então, os códigos <i>hash</i> deve refletir tal equivalência, isto é, se $x == y$, então $hash(x) == hash(y)$.


### Funções de compressão

Em geral, o código <i>hash</i> não é imediatamente útil como índice do vetor de <i>buckets</i> devido à
possibilidade de seu valor extrapolar os limites do vetor (para mais ou para menos). Portanto, é
necessário um passo adicional para garantir o mapeamento do código no intervalo $[0, N-1]$.

#### O método da divisão

Uma conversão simples é utilizar:
$$
h(k) = |k| mod N.
$$

A escolha de $N$ pode impactar na colisão. Idealmente, deseja-se que a probabilidade de duas chaves
serem mapeadas para o mesmo <i>bucket</i> seja $\frac{1}{N}$. Na prática, sugere-se escolher $N$ primo
para reduzir a chance de aparecem padrões que aumentam as colisões. Embora, isso não seja
garantidamente a melhor escolha pois podem haver padrões nos dados que prejudiquem tal escolha.

#### O método MSD (<i>MAD -- Multiply, add and divide</i>)

Um método mais sofisticado que ajuda a eliminar padrões repetidos nas chaves é o método MAD.

$$
h(k) = |ak + b| mod N.
$$

onde $N$ é escolhido como número primo e $a$ e $b$ são inteiros não negativos aleatoriamente
escolhidos no momento de criação da função de forma que $a mod N \neq 0$. 


## Gerenciamento de colisões

Embora haja um esforço para minimizar o número de colisões,
mecanismos para gerenciar a tabela <i>hash</i> quando elas ocorrem ainda são necessários. Particularmente,
as colisões dificultam as operações de busca, inserção (ou modificação) e remoção.

### Separate Chaining

Uma maneira simples e efetiva de lidar com colisões é ter cada /bucket/ $A[i]$ armazenando um
pequeno mapa $M_{i}$ implementado utilizando uma lista. Esse mapa armazena as entradas $(k, v)$ tais
que $h(k) = i$. 

Nesse caso, a abordagem de <i>separate chaining</i> delega para o mini-mapa do <i>bucket</i> correspondente a
realização da operação fundamental. 

Essa implementação simples é suficiente quando o tamanho das listas $M_{i}$ é pequeno. 

Assumindo que, em uma boa função hash, a probabilidade de uma chave ser mapeada para um <i>bucket</i>
qualquer é aproximadamente $\frac{1}{N}$, sendo $N$ o tamanho do vetor de <i>buckets</i>. Então, caso
haja $n$ entradas para seram adicionadas, cada <i>bucket</i> ficaria com o  $\lambda = \frac{n}{N}$ elementos. Esse
valor é conhecido como fator de carga da tabela <i>hash</i>. No pior caso, as funções de inserção, busca e remoção podem ser implementadas em $O(\lceil \frac{n}{N} \rceil)$. Embora, o tempo esperado seja
$O(1)$ se $n$ for $O(N)$.

Apesar de sua simplicidade, o <i>separate chaining</i> possui a desvantagem de necessitar de uma estrutura
de dados auxiliar (lista) para armazenar as entradas com as chaves que colidiram. 

### <i>Open addressing</i> (endereçamento aberto)

A estratégia de <i>open addressing</i> economiza espaço por não utilizar estruturas adicionais, embora seja
mais complexa. Ainda, essa estratégia requer que o fator de carga seja no máximo 1 e que as entradas
sejam armazenadas diretamente nas células do vetor de <i>buckets</i>. Há diversas variações dessa
estratégia.

#### <i>Probing</i> (sondagem) linear e variantes

Talvez a técnica mais simples de <i>probing linear</i>.  Nesse método, tenta-se inserir a entrada $(k, v)$
no <i>bucket</i> $A[i]$, caso o <i>bucket</i> esteja ocupado, então tenta-se inserir no <i>bucket</i> seguinte, de
maneira circular, até que seja possível inserir a entrada.

Note que, com essa estratégia, a busca por uma determinada chave precisa prosseguir até que seja
encontrada uma célula vazia. 

A operação <i>erase(k)</i> pode ser implementada marcando a célula removida com um caractere especial
indicando a disponibilidade da posição. 

A operação inserção deve se lembrar de posições disponíveis na busca por $k$, para que seja
realizada a inserção caso $k$ não seja encontrado. 

Dessa forma, o <i>probing linear</i> economiza espaço mas complica as operações elementares. Ainda, a
desvantagem de poluir faixas contínuas do vetor de <i>buckets</i>. Tais porçoes contínuas fazem com que as
buscas sejam mais custosas.

Variantes dessa estratégia consideram uma busca não linear, isto é, ao invés de verificar a posição
seguinte do vetor <i>bucket</i>, a posição a ser verificada é baseada em uma função da posição
atual. Duas opções conhecidas são utilizar uma função quadrática ou uma segunda função <i>hash</i>. 

O probing linear e suas variações são interessantes quando a memória é limitada, caso isso não seja
um problema, o <i>separate chaining</i> costuma ser mais indicado.


### Fator de carga e rehashing

Em geral, o fator de carga $\lambda = \frac{n}{N}$ deve ser mantido menor do que a
unidade. Experimentos sugerem que para a estratégia de <i>open addressing</i> deve-se manter $\lambda <
0.5$ e no caso de <i>separate chaining</i> $\lambda < 0.9$. 

No caso de um fator de carga alto, uma estratégia utilizada consiste em reinserir os elementos em
uma tabela <i>hash</i> maior (pelo menos o dobro do tamanho).

# Implementação de um dicionário com tabela <i>hash</i>

A implementação do dicionário considera a classe MapBase, definida anteriormente. É definida a classe "abstrata" <i>HashMapBase</i>, responsável por gerenciar o vetor de <i>buckets</i>. Assume-se que o código <i>hash</i> do objeto a ser usado como chave está definido e, então, utiliza-se a estratégia <i>multiply-add-divide</i> para mapeá-la dentro dos limites da lista de <i>buckets</i>. Note que as lógicas de busca, inserção e remoção dependem da implementação, apenas as lógicas mais gerais, referentes ao vetor de <i>buckets</i> são definidas.

In [ ]:
class HashMapBase(MapBase):
    """Abstract base class for map using hash-table with MAD compression.
    Keys must be hashable and non-None.
    """

    def __init__(self, cap=11, p=109345121):
        """Create an empty hash-table map.
        cap     initial table size (default 11)
        p       positive prime used for MAD (default 109345121)
        """
        self._table = cap * [None]
        self._n = 0                                   # number of entries in the map
        self._prime = p                               # prime for MAD compression
        self._scale = 1 + randrange(p-1)              # scale from 1 to p-1 for MAD
        self._shift = randrange(p)
    
    def _hash_function(self, k):
        return (hash(k)*self._scale + self._shift) % self._prime % len(self._table)
    
    def __len__(self):
        return self._n
    
    def _resize(self, c):
        """Resize bucket array to capacity c and rehash all items."""
        old = list(self.items())       # use iteration to record existing items
        self._table = c * [None]       # then reset table to desired capacity
        self._n = 0                    # n recomputed during subsequent adds
        for (k,v) in old:
            self[k] = v                  # reinsert old key-value pair

O gerenciamento das colisões é feito pela classe que implementa a classe abstrata, em particular os métodos <i>bucket\_getitem</i>, <i>bucket\_setitem</i> e <i>_bucket_delitem</i>..

In [ ]:
def __getitem__(self, k):
    j = self._hash_function(k)
    return self._bucket_getitem(j, k)             # may raise KeyError

def __setitem__(self, k, v):
    j = self._hash_function(k)
    self._bucket_setitem(j, k, v)                 # subroutine maintains self._n
    if self._n > len(self._table) // 2:           # keep load factor <= 0.5
        self._resize(2 * len(self._table) - 1)      # number 2^x - 1 is often prime

def __delitem__(self, k):
    j = self._hash_function(k)
    self._bucket_delitem(j, k)                    # may raise KeyError
    self._n -= 1

Para esta implementação, foi utilizada a estratégia <i>separate chaining</i> para lidar com as colisões, implementada na classe <i>ChainHashMap</i>. Então, é a classe <i>ChainHashMap</i>, filha da <i>HashMapBase</i> que implementa os métodos <i>_bucket_getitem</i>, <i>_bucket_setitem</i> e <i>_bucket_delitem</i>. Além do mecanismos de iteração.

In [ ]:
class ChainHashMap(HashMapBase):
    """Hash map implemented with separate chaining for collision resolution."""

    def _bucket_getitem(self, j, k):  
        bucket = self._table[j]
        if bucket is None:
            raise KeyError('Key Error: ' + repr(k))        # no match found
        return bucket[k]                                 # may raise KeyError

    def _bucket_setitem(self, j, k, v):
        if self._table[j] is None:
            self._table[j] = UnsortedTableMap()     # bucket is new to the table
        oldsize = len(self._table[j])
        self._table[j][k] = v
        if len(self._table[j]) > oldsize:         # key was new to the table
            self._n += 1                            # increase overall map size

    def _bucket_delitem(self, j, k):
        bucket = self._table[j]
        if bucket is None:
            raise KeyError('Key Error: ' + repr(k))        # no match found
        del bucket[k]                                    # may raise KeyError

    def __iter__(self):
        for bucket in self._table:
            if bucket is not None:                         # a nonempty slot
                for key in bucket:
                    yield key

## Implementação completa

In [2]:
from collections import MutableMapping
from random import randrange         # used to pick MAD parameters

class MapBase(MutableMapping):
    """Our own abstract base class that includes a nonpublic _Item class."""

    class _Item:
        """Lightweight composite to store key-value pairs as map items."""
        __slots__ = '_key', '_value'

        def __init__(self, k, v):
            self._key = k
            self._value = v

    def __eq__(self, other):               
        return self._key == other._key   # compare items based on their keys

    def __ne__(self, other):
        return not (self == other)       # opposite of __eq__

    def __lt__(self, other):               
        return self._key < other._key    # compare items based on their keys


class UnsortedTableMap(MapBase):
    """Map implementation using an unordered list."""

    def __init__(self):
        """Create an empty map."""
        self._table = []                              # list of _Item's
  
    def __getitem__(self, k):
        """Return value associated with key k (raise KeyError if not found)."""
        for item in self._table:
            if k == item._key:
                return item._value
        raise KeyError('Key Error: ' + repr(k))

    def __setitem__(self, k, v):
        """Assign value v to key k, overwriting existing value if present."""
        for item in self._table:
            if k == item._key:                          # Found a match:
                item._value = v                           # reassign value
                return                                    # and quit    
        # did not find match for key
        self._table.append(self._Item(k,v))

    def __delitem__(self, k):
        """Remove item associated with key k (raise KeyError if not found)."""
        for j in range(len(self._table)):
            if k == self._table[j]._key:                # Found a match:
                self._table.pop(j)                        # remove item
                return                                    # and quit    
        raise KeyError('Key Error: ' + repr(k))

    def __len__(self):
        """Return number of items in the map."""
        return len(self._table)

    def __iter__(self):                             
        """Generate iteration of the map's keys."""
        for item in self._table:
            yield item._key                             # yield the KEY


class HashMapBase(MapBase):
    """Abstract base class for map using hash-table with MAD compression.
    Keys must be hashable and non-None.
    """

    def __init__(self, cap=11, p=109345121):
        """Create an empty hash-table map.
        cap     initial table size (default 11)
        p       positive prime used for MAD (default 109345121)
        """
        self._table = cap * [None]
        self._n = 0                                   # number of entries in the map
        self._prime = p                               # prime for MAD compression
        self._scale = 1 + randrange(p-1)              # scale from 1 to p-1 for MAD
        self._shift = randrange(p)                    # shift from 0 to p-1 for MAD

    def _hash_function(self, k):
        return (hash(k)*self._scale + self._shift) % self._prime % len(self._table)

    def __len__(self):
        return self._n

    def __getitem__(self, k):
        j = self._hash_function(k)
        return self._bucket_getitem(j, k)             # may raise KeyError

    def __setitem__(self, k, v):
        j = self._hash_function(k)
        self._bucket_setitem(j, k, v)                 # subroutine maintains self._n
        if self._n > len(self._table) // 2:           # keep load factor <= 0.5
            self._resize(2 * len(self._table) - 1)      # number 2^x - 1 is often prime

    def __delitem__(self, k):
        j = self._hash_function(k)
        self._bucket_delitem(j, k)                    # may raise KeyError
        self._n -= 1

    def _resize(self, c):
        """Resize bucket array to capacity c and rehash all items."""
        old = list(self.items())       # use iteration to record existing items
        self._table = c * [None]       # then reset table to desired capacity
        self._n = 0                    # n recomputed during subsequent adds
        for (k,v) in old:
            self[k] = v                  # reinsert old key-value pair


class ChainHashMap(HashMapBase):
    """Hash map implemented with separate chaining for collision resolution."""

    def _bucket_getitem(self, j, k):  
        bucket = self._table[j]
        if bucket is None:
            raise KeyError('Key Error: ' + repr(k))        # no match found
        return bucket[k]                                 # may raise KeyError

    def _bucket_setitem(self, j, k, v):
        if self._table[j] is None:
            self._table[j] = UnsortedTableMap()     # bucket is new to the table
        oldsize = len(self._table[j])
        self._table[j][k] = v
        if len(self._table[j]) > oldsize:         # key was new to the table
            self._n += 1                            # increase overall map size

    def _bucket_delitem(self, j, k):
        bucket = self._table[j]
        if bucket is None:
            raise KeyError('Key Error: ' + repr(k))        # no match found
        del bucket[k]                                    # may raise KeyError

    def __iter__(self):
        for bucket in self._table:
            if bucket is not None:                         # a nonempty slot
                for key in bucket:
                    yield key

# Aplicação: O problema da soma de pares

O problema  da soma de pares, pode ser apresentado da seguinte forma. Dado um inteiro $k$ e uma lista $A$ de valores desordenados, encontre um par $\{x, y\} \subset A$ tal que $x + y = k$. Trata-se de um caso particular do problema de soma de subconjuntos (<i>subset sum problem</i>), que é NP-difícil. 

Uma abordagem inicial consiste em checar todos os pares explicitamente. Como é necessário percorrer todos os pares de elementos e há $\frac{n^2 - n}{2}$ pares, então o algoritmo é $O(n^2)$.

In [22]:
from random import seed, randint

def pair_sum1(k, A):
    for i, x in enumerate(A):
        for y in A[i:]:
            if x + y == k:
                return x, y
            
            
seed(0)
lb, ub = 0, 100000
max_nums = 100000
A = [randint(lb, ub) for _ in range(max_nums)]
k = 113         
%time pair_sum1(k, A)

CPU times: user 1.18 s, sys: 0 ns, total: 1.18 s
Wall time: 1.18 s


(9, 104)

Uma segunda opção consiste em, para cada $x$ buscar por $y=k-x$ no vetor. Dessa forma, a busca por $k-x$ pode ser feita percorrendo o vetor, embora isso também resulte em um algoritmo $O(n^2)$. No entanto, pode-se primeiramente ordenar o vetor $A$, permitindo a utilização de busca binária por $y=k-x$. Dessa forma, tem-se um algoritmo $O(nlogn)$.

Uma terceira opção (<i>pair\_sum2</i>, a seguir) consiste em utilizar uma tabela <i>hash</i> com $x$ sendo a chave e $y = k-x$ seu respectivo valor. Considere uma implementação utilizando <i>separate chaining</i>. Dessa forma, para cada $x \in A$, é necessário buscar por $y = k-x$ apenas no <i>bucket</i> identificado por $h(x)$. Tal <i>bucket</i> possui tamanho médio $\frac{n}{cap}$, em que $cap$ é o tamanho da lista de <i>buckets</i>. Se a capacidade $cap$ for $O(n)$, então a busca possui complexidade esperada $O(1)$. 

In [72]:
from random import seed, randint
from collections import MutableMapping
from random import randrange         # used to pick MAD parameters

class MapBase(MutableMapping):
    """Our own abstract base class that includes a nonpublic _Item class."""

  #------------------------------- nested _Item class -------------------------------
    class _Item:
        """Lightweight composite to store key-value pairs as map items."""
        __slots__ = '_key', '_value'

        def __init__(self, k, v):
            self._key = k
            self._value = v

    def __eq__(self, other):               
        return self._key == other._key   # compare items based on their keys

    def __ne__(self, other):
        return not (self == other)       # opposite of __eq__

    def __lt__(self, other):               
        return self._key < other._key    # compare items based on their keys


class UnsortedTableMap(MapBase):
    """Map implementation using an unordered list."""

    def __init__(self):
        """Create an empty map."""
        self._table = []                              # list of _Item's
  
    def __getitem__(self, k):
        """Return value associated with key k (raise KeyError if not found)."""
        for item in self._table:
            if k == item._key:
                return item._value
        raise KeyError('Key Error: ' + repr(k))

    def __setitem__(self, k, v):
        """Assign value v to key k, overwriting existing value if present."""
        for item in self._table:
            if k == item._key:                          # Found a match:
                item._value = v                           # reassign value
                return                                    # and quit    
        # did not find match for key
        self._table.append(self._Item(k,v))

    def __delitem__(self, k):
        """Remove item associated with key k (raise KeyError if not found)."""
        for j in range(len(self._table)):
            if k == self._table[j]._key:                # Found a match:
                self._table.pop(j)                        # remove item
                return                                    # and quit    
        raise KeyError('Key Error: ' + repr(k))

    def __len__(self):
        """Return number of items in the map."""
        return len(self._table)

    def __iter__(self):                             
        """Generate iteration of the map's keys."""
        for item in self._table:
            yield item._key                             # yield the KEY


class HashMapBase(MapBase):
    """Abstract base class for map using hash-table with MAD compression.
    Keys must be hashable and non-None.
    """

    def __init__(self, cap=11, p=109345121):
        """Create an empty hash-table map.
        cap     initial table size (default 11)
        p       positive prime used for MAD (default 109345121)
        """
        self._table = cap * [None]
        self._n = 0                                   # number of entries in the map
        self._prime = p                               # prime for MAD compression
        self._scale = 1 + randrange(p-1)              # scale from 1 to p-1 for MAD
        self._shift = randrange(p)                    # shift from 0 to p-1 for MAD

    def _hash_function(self, k):
        return (hash(k)*self._scale + self._shift) % self._prime % len(self._table)

    def __len__(self):
        return self._n

    def __getitem__(self, k):
        j = self._hash_function(k)
        return self._bucket_getitem(j, k)             # may raise KeyError

    def __setitem__(self, k, v):
        j = self._hash_function(k)
        self._bucket_setitem(j, k, v)                 # subroutine maintains self._n
        if self._n > len(self._table) // 2:           # keep load factor <= 0.5
            self._resize(2 * len(self._table) - 1)      # number 2^x - 1 is often prime

    def __delitem__(self, k):
        j = self._hash_function(k)
        self._bucket_delitem(j, k)                    # may raise KeyError
        self._n -= 1

    def _resize(self, c):
        """Resize bucket array to capacity c and rehash all items."""
        old = list(self.items())       # use iteration to record existing items
        self._table = c * [None]       # then reset table to desired capacity
        self._n = 0                    # n recomputed during subsequent adds
        for (k,v) in old:
            self[k] = v                  # reinsert old key-value pair


class ChainHashMap(HashMapBase):
    """Hash map implemented with separate chaining for collision resolution."""

    def _bucket_getitem(self, j, k):  
        bucket = self._table[j]
        if bucket is None:
            raise KeyError('Key Error: ' + repr(k))        # no match found
        return bucket[k]                                 # may raise KeyError

    def _bucket_setitem(self, j, k, v):
        if self._table[j] is None:
            self._table[j] = UnsortedTableMap()     # bucket is new to the table
        oldsize = len(self._table[j])
        self._table[j][k] = v
        if len(self._table[j]) > oldsize:         # key was new to the table
            self._n += 1                            # increase overall map size

    def _bucket_delitem(self, j, k):
        bucket = self._table[j]
        if bucket is None:
            raise KeyError('Key Error: ' + repr(k))        # no match found
        del bucket[k]                                    # may raise KeyError

    def __iter__(self):
        for bucket in self._table:
            if bucket is not None:                         # a nonempty slot
                for key in bucket:
                    yield key
   
        
def pair_sum2(k, A):    
    hash_table = ChainHashMap(cap=10)
    for x in A:
        y = k - x
        if y in hash_table:
            return x, y
        else:
            hash_table[x] = y

seed(0)
lb, ub = 0, 100000
max_nums = 100000
A = [randint(lb, ub) for _ in range(max_nums)]
k = 113         
%time print(pair_sum1(k, A))
%time print(pair_sum2(k, A))

(9, 104)
CPU times: user 1.16 s, sys: 0 ns, total: 1.16 s
Wall time: 1.16 s
(54, 59)
CPU times: user 54.5 ms, sys: 0 ns, total: 54.5 ms
Wall time: 54.3 ms


# Exercícios
1. Discuta a importância de códigos <i>hash</i> de um objeto serem imutáveis. Pense na inserção e na futura busca do objeto em uma tabela <i>hash</i>
2. Considere o problema da soma de pares, implemente a versão que ordena o vetor $A$ e, então, utiliza busca binária para encontrar $y$. Através de um exemplo, compare a sua eficiência com as outras versões apresentadas.
3. O recorte de código a seguir é relacionado à implementação da classe <i>ProbeHashMap</i> que implementa a estratégia de sondagem linear para gerenciar as colisões. Complete o código adequadamente.

In [ ]:
class ProbeHashMap(HashMapBase):
    """Hash map implemented with linear probing for collision resolution."""
    _AVAIL = object()       # sentinal marks locations of previous deletions

    def _is_available(self, j):
        """Return True if index j is available in table."""
        pass
        

    def _find_slot(self, j, k):
        """Search for key k in bucket at index j.
        Return (success, index) tuple, described as follows:
        If match was found, success is True and index denotes its location.
        If no match found, success is False and index denotes first available slot.
        """
        pass

    def _bucket_getitem(self, j, k):
        #...
        if not found:
            raise KeyError('Key Error: ' + repr(k))        # no match found
        #...

    def _bucket_setitem(self, j, k, v):
        pass

    def _bucket_delitem(self, j, k):
        #...
        if not found:
            raise KeyError('Key Error: ' + repr(k))        # no match found
        self._table[s] = ProbeHashMap._AVAIL             # mark as vacated
        #...

    def __iter__(self):
        for j in range(len(self._table)):                # scan entire table
            if not self._is_available(j):
                yield self._table[j]._key


# Referências

- Material do Prof. Dr. Mário Felice. http://www2.dc.ufscar.br/~mario/ensino/2019s2/aed1/aed1.php
- Goodrich, Michael T., Roberto Tamassia, and Michael H. Goldwasser. Data structures and algorithms in Python. John Wiley & Sons Ltd, 2013.
- Bento, Lucila Maria de Souza, Vinícius Gusmão Pereira de Sá, and Jayme Luiz Szwarcfiter. "Some Illustrative Examples on the Use of Hash Tables." Pesquisa Operacional 35.2 (2015): 423-437.